# Modeling Holidays and Special Events

If you have holidays or other recurring events that you’d like to model, you must create a dataframe for them. It has two columns (`event` and `ds`) and a row for each occurrence of the holiday. It must include all occurrences of the holiday, both in the past (back as far as the historical data go) and in the future (out as far as the forecast is being made). If they won’t repeat in the future, Prophet will model them and then not include them in the forecast.

Events can be added as either additive or multiplicative components.

Windows can additionally be defined to include days before/after the event.

We will use the time series of the log daily page views for the Wikipedia page for Peyton Manning as an example.
First, we load the data:

In [1]:
import pandas as pd
from neuralprophet import NeuralProphet

In [2]:
df = pd.read_csv('../example_data/wp_log_peyton_manning.csv')

Here we create a dataframe that includes the dates of all of Peyton Manning’s playoff appearances, one for past events and one for future events:

In [3]:
## user specified events
# history events
playoffs_history = pd.DataFrame({
        'event': 'playoff',
        'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                              '2010-01-24', '2010-02-07', '2011-01-08',
                              '2013-01-12', '2014-01-12', '2014-01-19',
                              '2014-02-02', '2015-01-11', '2016-01-17']),
    })

superbowls_history = pd.DataFrame({
    'event': 'superbowl',
    'ds': pd.to_datetime(['2010-02-07', '2014-02-02']),
    })

history_events_df = pd.concat((playoffs_history, superbowls_history))

# future values of events
playoffs_future = pd.DataFrame({
    'event': 'playoff',
    'ds': pd.to_datetime(['2016-01-24', '2016-02-07'])
})

superbowl_future = pd.DataFrame({
    'event': 'superbowl',
    'ds': pd.to_datetime(['2016-02-07'])
})

future_events_df = pd.concat((playoffs_future, superbowl_future))

In [4]:
## user specified events
# history events
playoffs = pd.DataFrame({
    'event': 'playoff',
    'ds': pd.to_datetime([
        '2008-01-13', '2009-01-03', '2010-01-16',
        '2010-01-24', '2010-02-07', '2011-01-08',
        '2013-01-12', '2014-01-12', '2014-01-19',
        '2014-02-02', '2015-01-11', '2016-01-17',
        '2016-01-24', '2016-02-07',
    ]),
})

superbowls = pd.DataFrame({
    'event': 'superbowl',
    'ds': pd.to_datetime([
        '2010-02-07', '2014-02-02', '2016-02-07'
    ]),
})

events_df = pd.concat((playoffs, superbowls))

Above we have included the superbowl days as both playoff games and superbowl games. This means that the superbowl effect will be an additional additive bonus on top of the playoff effect.

##  Additive Events

Once the table is created, event effects are included in the model by adding them in with the `add_events` function.

By default, events are modelled as additive events. Here, we model both playoffs as additive events.

In [5]:
# NeuralProphet Object
m = NeuralProphet()

# set the model to expect these events
m = m.add_events(["playoff", "superbowl"])

# create the data df with events
history_df = m.create_df_with_events(df, events_df)

In [6]:
events = history_df[(history_df['playoff'].abs() + history_df['superbowl'].abs()) > 0]
events

,ds,y,playoff,superbowl
34,2008-01-13,9.996522,1.0,0.0
361,2009-01-03,9.121946,1.0,0.0
730,2010-01-16,8.779404,1.0,0.0
750,2010-02-07,11.191452,1.0,1.0
1076,2011-01-08,9.098067,1.0,0.0
1806,2013-01-12,10.541862,1.0,0.0
2169,2014-01-12,10.693308,1.0,0.0
2176,2014-01-19,11.503622,1.0,0.0
2190,2014-02-02,11.760520,1.0,1.0
2532,2015-01-11,9.886596,1.0,0.0


In [7]:
# fit the model
metrics = m.fit(history_df, freq="D")

INFO: nprophet.utils - set_auto_seasonalities: Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[40/40]: 100%|██████████| 40/40 [00:03<00:00, 10.91it/s, SmoothL1Loss=0.00569, MAE=0.368, RegLoss=0]


In [8]:
# forecast with events
future = m.make_future_dataframe(
    df=history_df, 
    events_df=events_df, 
    periods=30, 
    n_historic_predictions=len(df)
)
forecast = m.predict(df=future)

TypeError: argument of type 'NoneType' is not iterable

The events effect can be seen in the forecast dataframe:

In [ ]:
events = forecast[(forecast['event_playoff'].abs() + forecast['event_superbowl'].abs()) > 0]
events


The events effects will also show up in the plots, where we see that there is a spike on the days around playoff appearances, with an especially large spike for the superbowl:

In [ ]:
## plotting forecasts
fig = m.plot(forecast)

In [ ]:
## plotting components
fig_comp = m.plot_components(forecast)

In [ ]:
## plotting parameters
fig_param = m.plot_parameters()

## Event Windows
You can also include the arguments `lower_window` and `upper_window` which extend the holiday out to `[lower_window, upper_window]` days around the date. 

For instance, if you wanted to include Christmas Eve in addition to Christmas you’d include `lower_window=-1, upper_window=0`. If you wanted to use Black Friday in addition to Thanksgiving, you’d include `lower_window=0, upper_window=1`. 

In [ ]:
m = NeuralProphet()

# set event configs to NeuralProphet object with windows
m = m.add_events(["playoff"], lower_window=-1, upper_window=1)
m = m.add_events(["superbowl"])

# create the data df with events
history_df = m.create_df_with_events(df, events_df)

# fit the model
metrics = m.fit(history_df, freq="D")

In [ ]:
# make future dataframe with events known in future
future = m.make_future_dataframe(df=history_df, events_df=events_df, periods=365, n_historic_predictions=len(df))
forecast = m.predict(df=future)

In [ ]:
## plotting parameters
fig = m.plot(forecast)
fig_param = m.plot_parameters()

## Multiplicative Events

In [ ]:
m = NeuralProphet()

# set event configs to NeuralProphet object with windows
m = m.add_events(["playoff"])
m = m.add_events(["superbowl"], mode="multiplicative")

# create the data df with events
history_df = m.create_df_with_events(df, events_df)

# fit the model
metrics = m.fit(history_df, freq="D")

In [ ]:
# make future dataframe with events known in future
future = m.make_future_dataframe(history_df, events_df, periods=30, n_historic_predictions=len(df))
forecast = m.predict(df=future)

In [ ]:
## plotting components
fig_comp = m.plot(forecast)

In [ ]:
# plot parameters
fig_param = m.plot_parameters()

## Country Specific Holidays

In [ ]:
m = NeuralProphet()

# add the country specific holidays
m = m.add_country_holidays("US", mode="additive")

# fit the model
metrics = m.fit(df, freq="D")

In [ ]:
# make future dataframe with events known in future
future = m.make_future_dataframe(df=df, periods=30, n_historic_predictions=len(df))
forecast = m.predict(df=future)

In [ ]:
## plotting components
fig_param = m.plot_parameters()